# [Crowdflower Search Results Relevance](https://www.kaggle.com/c/crowdflower-search-relevance)

Kaggle에서 제공하는 Crowdflower Search Results Relevance data를 사용하여 검색 서비스 만족도 판별 모델을 개발한 과정을 정리해 봤습니다.  
(분석 과정은 preprocessing.ipynb, LSA.ipynb 파일을 따르고 jupyter nootbook내에서 사용되는 함수는 utility module로 구현되어 있습니다. 모든 과정은 preprocessing.py와 model.py로 실행 가능합니다.)

index
+ [설명](#설명)  
+ [TF-IDF](#TF-IDF)
  + [전처리](#전처리)  
  + [유사도](#유사도)
+ [모델링](#모델링)
  + [평가 지표](#평가지표)
  + [LogisticRegression](#LogisticRegression)
  + [LSA](#LSA)
  + [SVM](#SVM)
+ [모델링 결과 해석](#모델링결과해석)
  + [문제점](#문제점)
  + [개선안](#개선안)
    + [높은 분산과 오버피팅](#높은분산과오버피팅)
    + [비대칭 데이터](#비대칭데이터)
+ [샘플링](#샘플링)
  + [추가 평가 지표](#추가평가지표)
+ [그리드서치](#그리드서치)
+ [최종 모델](#최종모델)
+ [결론](#결론)

## 설명

사용자가 서비스를 이용하면서 얻는 만족도는 검색 시스템의 정확도에 크게 영향을 받습니다. 예를 들어 고양이를 검색했는데 결과로 스시가 나온다면 사용자는 전체 서비스를 경험하기도 전에 서비스가 엉터리라고 느낍니다.

kaggle에 따르면 소규모 온라인 서비스를 제공하는 소기업에서는 검색 알고리즘의 성능을 평가하는 좋은 방법이 없어 사용자에게 좋은 서비스를 제공하기가 어렵다고 합니다. 

이런 이유로 사용자가 검색 서비스를 이용할 때 검색 결과를 보고 얼마나 만족할지 판별하는 모델이 필요합니다.
추가로 만족도를 판별할 수 있다면 사용자가 만족스럽지 못한 결과를 얻었을때 모델을 기반으로 만족도가 높을것으로 예측되는 결과를 추천해줄 수 도 있습니다.

---

어떻게 사용자의 만족도를 판단할 수 있을까요? 먼저 사용자의 입장이 되어 어떤 경우에 서비스가 만족스러울지 생각해 보았습니다. 아주 당연하게도 검색어를 입력했을때 나온 결과가 얻고자 했던 정보라면 만족스러울 것이고 상관없는 정보라면 불만족스러울 것입니다.

그렇다면 사용자가 얻고자 하는 정보가 무었일까요? 사용자가 검색 서비스를 이용하는 흐름을 생각해본다면 쉽게 생각할 수 있습니다. 사용자는 검색 서비스를 이용할 때 얻고자 하는 정보와 최대한 비슷하게 검색어를 입력합니다. 즉, 사용자가 얻고자 하는 정보란 검색어와 **유사도**(비슷한 정도)가 높은 결과라고 생각했습니다.

이제 1차 목표는 검색어와 검색결과의 유사도를 측정하는 것이 되었습니다.

## TF-IDF

자연어 처리 분야에서 문서의 유사도는 **TF-IDF**라는 방법을 사용하여 구합니다. TF-IDF는 여러 문서로 이루어진 문서집합이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는지 계산하는 방법입니다.

간단히 TF(Term Frequency)라는 단어의 빈도에 IDF(Inverse Document Frequency)라고 하는 역 문서 빈도를 가중치로 곱하여 문서의 각 단어들마다 중요한 정도를 수치화 하는 방법입니다.(TF-IDF에 대한 코드는 model.py에 구현되어 있습니다.)

---
**TF-IDF 설명**


좀더 자세하게, 수 식으로는 TF-IDF $= TF \times IDF = TF \times \log\left(\frac{n}{1+DF}\right)$ 입니다. 여기서 TF란 각 문서에서 등장한 단어의 빈도수를 뜻하며 TF(문서, 단어)로 표시합니다. 예를들어

>문서1 : 우리집 귀여운 고양이  
 문서2 : 강아지 고양이는 귀엽다  
 문서3 : 고양이는 강아지보다 귀엽다  
 문서4 : 우리집 고양이 쿤이는 귀여운 고양이다  

이라는 문서 집합이 있을때 TF(문서1, 고양이) = 1, TF(문서4, 고양이) = 2 가 됩니다.

IDF $\log\left(\frac{n}{1+DF}\right)$는 단어의 가중치를 의미하며 DF의 역수에 총 문서의 수 n을 곱하고 로그를 취한 수 입니다. 여기서 DF란 각 단어들이 등장한 문서의 수 이며 DF(단어)로 표시합니다. 위의 예에서 DF(강아지)=2가 됩니다. 단어가 등장한 문서의 수를 의미하기 때문에 고양이라는 단어가 전체 문서에서 5번 등장 했지만 고양이가 등장한 문서의 수인 DF(고양이)=4가 됩니다. DF의 역수는 자주 등장하는 단어의 가중치는 낮게 설정하고 조금 등장하는 단어의 경우에는 가중치를 크게 줍니다. 예를 들어 '고양이'라는 단어는 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 예시의 경우는 상투어가 되어 가중치가 낮아집니다.
    
다시 위의 예에서 고양이라는 단어는 모든 문서에 등장하여 문서간 유사도를 계산하고자 할때 별 도움이 되지 않습니다. 수치로 계산해보면 $\log\left(\frac{4}{1+4}\right)$입니다. 두번 등장한 강아지($\log\left(\frac{4}{1+2}\right)$)와 비교해봤을때 작은 수치가 됩니다.

수식에서 분모에 1을 더하는 이유는 분모가 0이 되는 경우를 방지하고 단어 등장횟수의 스무딩 역할도 하기 위함 입니다.
마지막 $\log$를 취하는 이유는 보통 자연어 처리에서 사용되는 총문서의 수는 **log scale** 이기 때문입니다. 
지금 분석하고자 하는 데이터의 경우에도 **train set : 10158문서, test set : 22513문서** 이기 때문에 $\log$를 취하지 않으면 가중치가 log scale로 커져버리게 됩니다.

간단하게 위의 예시문장으로 TF-IDF를 구해보겠습니다. 

각 문서에 등장하는 단어들의 빈도(TF)를 행렬로 표현하면 한눈에 보기쉽습니다. 이를 DTM(Document-Term Matrix)표기법이라고 합니다. 
       
|     |귀여|운|고양이|는|강아지|우리집|쿤이|보다|다|
|-----|----|--|-----|--|-----|------|---|---|---|
|문서1|1   |1 |1    |0 |0     |1     |0  |0  |0 |
|문서2|1   |0 |1    |1 |1     |0     |1  |0  |1 |
|문서3|1   |0 |1    |1 |1     |0     |0  |1  |1 |
|문서4|1   |1 |2    |1 |0     |1     |1  |0  |1 |

DTM행렬에 각 단어들의 가중치(IDF)를 곱해주면 TF-IDF를 구할수 있습니다.  
IDF(귀여) = IDF(고양이) = $\log\left(\frac{4}{1+4}\right) = \log\frac{4}{5} = -0.223$ 이 나옵니다. 음수는 단어가 모든 문서에 등장할 때 나오는데, 모든 문서에 등장하는 단어는 문서의 특징을 잘 표현하지 못하기 때문에 $0$으로 바꿔 진행합니다.  
IDF(는) = IDF(다) = $\log\left(\frac{4}{1+3}\right) = 0$  
IDF(운) = IDF(강아지) = IDF(우리집) = $\log\left(\frac{4}{1+2}\right) = 0.287$      
IDF(쿤이) = IDF(보다) $\log\left(\frac{4}{1+1}\right) = 0.693 $ 

|     |귀여   |운   |고양이|는|강아지|우리집|쿤이 |보다  |다|
|---- |------|-----|------|--|-----|------|-----|-----|--|
|문서1|0      |0.287|0    |0 |0    |0.287|0    |0    |0 |
|문서2|0      |0    |0    |0 |0.287|0     |0    |0    |0 |
|문서3|0      |0    |0    |0 |0.287|0     |0    |0.693|0 |
|문서4|0      |0.287|0    |0 |0    |0.287 |0.693|0    |0 |

이렇게 구한 TF-IDF행렬의 각row는 문서의 수치화된 벡터로 볼수 있습니다.
각 벡터로 부터 유사도를 계산할 수 있습니다.

---

데이터에 TF-IDF를 적용하기 전에 자연어 처리 분야에서 전처리는 모델의 성능을 좌우할만큼 중요한 작업입니다. 특히 TF-IDF에서는 모든 단어들의 빈도를 참고하기 때문에 정제, 정규화 작업을 거친 토큰화 과정이 매우 중요합니다.


**TF-IDF 코드**

sklearn 의 [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)를 사용하여 TF-IDF를 적용합니다.
다음은 TfidfVectorizer 세부 설정입니다.  
+  min_df=3 : 빈도수가 3번 이하인 단어 무시  
+  max_features=None : 단어 빈도수의 맥시멈 설정 x
+  strip_accents='unicode' : 모든 특수문자와 pattern을 정재할수 없으므로 unicode사용       
+  analyzer='word' : corpus가 문장 list로 이루어져있어 단어단위로 분석  
+  token_pattern=r'\w{1,}' : analyzer == 'word'일 때 token_pattern을 기반으로 단어 인식   
+  ngram_range=(1, 3) : word window의 범위 1~3, 약 90% 쿼리가 word size 3이하로 이루어짐  
+  use_idf=True : idf사용   
+  smooth_idf=True : 분모가 0이 되는것을 방지  
+  sublinear_tf=True : 단어 스무딩, replace 1 + log(tf).   
+  stop_words : 불용어 제거  

```python
tfv = text.TfidfVectorizer(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word', \
                           token_pattern=r'\w{1,}', ngram_range=(1, 3), use_idf=True, smooth_idf=True, \
                           sublinear_tf=True, stop_words = stop_words)

full_data = train_query + train_title
tfv.fit(full_data)

X_train = hstack([tfv.transform(train_query), tfv.transform(train_title)])
```

### 전처리

kaggle에서 제공한 데이터는 쿠팡과 같은 쇼핑몰에서 사용자가 검색한 쿼리와 검색결과인 제품과 설명이 제공됩니다. 따라서 코퍼스에는 가격, 제품 홈페이지, 특정 제품코드등의 정보가 많습니다. 하지만 사용자의 검색정보인 쿼리 데이터에서는 가격, 제품 홈페이지들을 직접 입력으로 사용하지 않았기 때문에 이러한 특정 패턴들을 지워야 합니다.

이미지는 kaggle에서 제공한 데이터의 첫 4개의 row입니다. columns에서 query가 검색어 이고 product_title, product_description이 제품명과 설명입니다. target인 만족도는 median_relevance입니다. 

<img src=".\img\train.PNG" width=90%> 

먼저 모든 단어에 대하여 대,소문자를 통합한 후 정규표현식을 사용하여 이메일, URL, HTML tag등의 텍스데이터를 정제했습니다. 결과로 LED-led, Levis-levis처럼 동일한 뜻이지만 다르게 표현되었던 단어들이 같은 단어가 되었고 유사도 측정에 의미가 없지만 빈도수는 낮아 가중치가 높게 나올것으로 예상되는 특정 단어들(이메일, 홈페이지 URL등)이 정제 되었습니다.

TF-IDF는 모든 단어들의 빈도를 사용하기 때문에 불용어 처리 또한 중요합니다. 불용어란 this, i, we, the등으로 자주 사용되지만 문서의 유사도 측정에는 도움이 되지 않는 단어들입니다. 간단히 nltk와, sklearn에서 제공하는 불용어 리스트와 의미없다고 생각되는 단어들을 추려 제거 하였습니다.

텍스트 데이터를 정제한 후에는 어간/표제어 추출을 통해 정규화를 해야합니다.
어간/표제어 추출은 watched-watch, running-run, is-be 처럼 다른 형태이지만 같은의미인 단어들을 하나의 단어로 표현하여 전체 단어의 수를 줄이는 작업입니다. 위의 예에서 보면 귀엽다 $\rightarrow$ 귀엽(어간)/다(어미), 귀엽 $\rightarrow$ 귀여(표제어)로 이해할 수 있다.

어간/표제어 추출은 nltk에서 제공하는 PorterStemmer와 WordNetLemmatizer를 사용하였습니다.
nltk는 어간추출을 위한 여러가지 알고리즘을 제공하는데 그중 PorterStemmer의 정확도가 높아 영어 자연어 처리에서 어간 추출을 하고자 한다면 가장 준수한 선택이라고 합니다.

다음 이미지는 전치리후의 데이터 입니다.  

<img src=".\img\train_preprocessed.PNG" width=90%> 

### 유사도

이제 검색어와 검색결과의 TF-IDF행렬을 통해 유사도를 측정해보겟습니다. 문서의 유사도 측정에는 여러가지 메트릭이 있는데, 현재 데이터 특성에 거리를 기반으로한 **유클리드 유사도** 보다 방향과 공통점을 기준으로 하는 **코사인**, **자카드** 유사도가  더욱 의미있어 유클리드 유사도는 사용하지 않습니다.(유사도를 구하는 코드는 utility.utility.py에 metric과 similarlity_stack class로 구현되어 있습니다.)

---

**유클리드 거리 유사도 설명**

다차원 공간에서 유클리드 거리란 같은 차원의 원소당 절대거리 제곱합의 제곱근으로 피타고라스 정리의 확장으로 볼수 있습니다.  

$$Euclidean\ distance(p, q) = \sqrt{\Sigma^{n}_{i=1}(p_i-q_i)^2} = \sqrt{(p_1-q_1)^2 + (p_2-q_2)^2 +\dots +(p_n-q_n)^2 }\ s.t \ p=(p_1, p_2, \dots, p_n), q=(q_1, q_2, \dots, q_n)$$

수식을 해석해보면 각 단어의 빈도수 차이 합을 유사도로 이용합니다. 이는 계산이 쉽고 직관적인 해석이 가능하지만 같은 의미를 가진 문장에서 단어의 빈도수에 따라 유사도가 달라지는 단점이 존재합니다.  
즉, 각 문서의 길이가 다른 코퍼스에서 유사도를 구할때 좋지못한 결과를 낼 수 있습니다. 

**코사인 유사도 설명**  

벡터로 수치화된 문서가 가르키는 방향이 얼마나 유사한지를 나타내는 척도 입니다.

$$ Cos\ similarity(p, q) = \frac{p \cdot q}{\lVert p \rVert \times \lVert p \rVert} = \frac{\Sigma^{n}_{i=1}p_i \times q_i}{\sqrt{\Sigma^{n}_{i=1}(p_i)^2} \times \sqrt{\Sigma^{n}_{i=1}(q_i)^2}} $$

절대적인 빈도수의 차이가 아니라 방향을 기준으로 하기에 유클리드 거리 유사도의 단점인 문서의 길이로 인한 오차를 보완하여 비교적 공정한 척도입니다.

**자카드 유사도 설명**

비교하고자 하는 문서들의 단어들을 집합으로 보고 공통적으로 등장하는 단어의 비율이 얼마나 유사한지 나타내는 척도 입니다.  

$$Jaccard\ similarity(p, q) = \frac{\left\vert p \cap q \right\vert}{\left\vert p \cup q \right\vert} \ $$

문서를 단어들의 집합으로 보기 때문에 같은 단어가 여러번 등장했을때 유클리드, 코사인 유사도가 달라지는 단점을 보완할 수 있습니다.

---  

사용자의 만족도 별로 검색어-결과 쌍의 코사인, 자카드 유사도 평균을 구해 보았습니다.  

|         |cos_sim mean|jaccard_sim mean|
|---------|------------|----------------|
|rating 1 |0.131505    |0.088091        |
|rating 2 |0.224782    |0.154311        |
|rating 3 |0.284875    |0.198081        |
|rating 4 |0.357709    |0.250337        |

전체적으로 유사도가 크지는 않지만 만족도 별로 차이가 있는걸로 보아 분류 모델링을 하는데 좋은 정보가 될것이라 생각 됩니다.

## 모델링
### 평가지표 

모델링을 할때는 목표로 하는 수치를 명확히 하여야 합니다. kaggle에서 제시한 평가 지표는 **quadratic weighted kappa** 입니다. 사용자가 남긴 만족도와 모델로 판단한 만족도가 일치할 확률과 우연하게 일치할 확률을 뺀후 가중치를 주어 0~1사이값으로 수치화 한 값입니다. 보통 0.4 ~ 0.6 : 적당한 일치도, 0.6 ~ 0.8 : 상당한 일치도, 0.8 ~ 1.0 : 완벽한 일치도로 봅니다. 검색만족도 평가의 특성이 잘 표현되는 지표임으로 그대로 사용하기로 합니다.  
(kappa score를 구하는 코드는 kaggle [Quadratic Kappa Metric explained in 5 simple steps](https://www.kaggle.com/aroraaman/quadratic-kappa-metric-explained-in-5-simple-steps)에 잘 구현되어 있어 조금 수정하여 사용하기로 했습니다. sklearn의 [cohen_kappa_score(weights='Quadratic')](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html)을 사용하여도 됩니다.)

### LogisticRegression

먼저 가장 간단한 분류모델중 하나인 [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)을 사용해봤습니다. TF-IDF를 그대로 사용하여 분류를 시도한 모델에서 **0.51197**점, 유사도를 추가한 모델에서 **0.56177**점으로 증가하였습니다. 생각했던 것 처럼 검색어와 검색결과의 유사도는 좋은 정보가 되었습니다.

**LogisticRegression 코드**
    
```python
lr = LogisticRegression()

X_train_sim = hstack([X_train, X_sim_train])
X_dev_sim = hstack([X_dev, X_sim_dev])

lr.fit(X_train, y)
# kappa score = 0.51197
print(metric.quadratic_weighted_kappa(y_dev, lr.predict(X_dev_sim))

lr.fit(X_train_sim, y)
# kappa score = 0.56177 
print(metric.quadratic_weighted_kappa(y_dev, lr.predict(X_dev_sim))                 
```
    
유사도가 성능을 높혔지만 아직 좋은 모델은 아닙니다. TF-IDF로 수치화된 벡터는 분류모델에게 단어의 의미까지 전달하지 못하기 때문입니다. 이는 빈도수를 기반으로 하는 TF-IDF가 필연적으로 만드는 희소행렬이 원인이라 생각되어 **SVD**를 통한 **LSA**를 사용하기로 했습니다. (희소 행렬이란 행렬의 값이 대부분 0인 경우를 가리키는 표현입니다.)

### LSA

텍스트 데이터는 여러가지 의미를 포함하고 있는경우가 많습니다. '고양이'라는 텍스트는 동물의 한 종류라는 의미와 함께 애완동물, 귀엽다 라는 의미도 내포하고 있습니다. 이러한 의미를 몇개의 문서에 몇번 등장했는지 만으로 파악하기란 쉽지 않습니다. 특히 TF-IDF는 문서와 단어가 많을수록 단어 하나하나가 가지는 정보가 옅어집니다. 1억개의 문서에 100만개가 넘는 단어가 있다고 생각해보십시오. TF-IDF행렬의 차원은 1억 * 100만이 되고 각 단어가 나타낼수 있는 문서의 정보 평균은 0.000001%가 됩니다.

**LSA(Latent symantic Analysis)**는 이런 문제점을 개선하기 위해 **SVD**를 사용하여 차원을 축소시키고, 단어들이 가지는 잠재적인 의미를 끌내낸다는 아이디어를 가지고 있습니다. 적용은 간단하지만 데이터가 클 경우 효과가 좋은 방법입니다.

---

**SVD, LSA 설명**
    
SVD(Singular Value Decomposition)를 이해하기 위해서는 선형대수학에 대한 배경지식이 필요로 합니다. 여기서는 수학적인 성질이나 자세한 증명은 넘어가고 간략한 설명과 왜, 어떻게 사용하는지에 초점을 맞추겟습니다.
    
SVD는 하나의 행렬을 3개의 행렬로 분해하면서 중요한 요소를 상위에 두는 방법입니다. 여기서 중요한 요소란 분해전 행렬의 정보를 많이 가진 요소입니다. 현제 데이터에 적용한다면 TF-IDF행렬을 3개의 행렬로 분해한뒤 중요한 요소를 내림차순으로 정렬한다고 생각하면 됩니다.

SVD가 LSA에 쓰이는 이유는 차원 축소입니다. **truncated SVD**라고 하며 상위에둔 중요한 요소중 n개를 잘라낸 형태입니다. 이렇게 하면 원래 행렬 차원이 상당히 줄어들지만 중요한 요소를 남기기 때문에 원래의 행렬의 정보를 보존할 수 있습니다. 또한 차원이 줄어들면서 중요하지 않은 정로를 삭제한다는 의의도 있습니다. 

LSA는 **co-occurrence**(동시 발생) 정보를 이용하여 단어의 의미를 파악합니다. co-occurrence란 코퍼스에서 두 단어가 동시에 등장한다는 의미입니다. 예를들어 새로운 문장5"해이는 고양이다"가 추가 되었을 때, TF-IDF는 새로운 단어 "해이"를 보고 빈도수와 가중치를 계산할 뿐 단어가 가지는 의미까지 파악하는것은 불가능 합니다. 반면 LSA는 "해이"와 함께 등장한 단어 "고양이"가 가지는 의미와 연관지어 파악합니다. 간단히 "고양이"-"귀엽다" 쌍이 자주 등장하였으므로 "해이"-"귀엽다"라는 의미를 유추하리라 예상할 수 있습니다.
    
LSA가 어떻게 co-occurrence정보를 사용하는지는 나누어진 행렬의 의미를 파악해야 합니다. 여기서는 차원이 축소되는 과정에서 co-occurrence성질을 가진 문서나 단어는 가까운 위치로 접근하게 된다고만 적겠습니다. 

적용방법은 간단합니다. truncated SVD를 하기위한 차원의 수를 정한뒤 나누어진 행렬중 일부를 곱해주면 의미를 포함한 유사도를 구할수 있습니다. 여기서 차원은 분류하고자 하는 문서들의 종류(토픽)라고 생각하면 됩니다. 검색어의 갯수와 비슷한 250으로 설정해보겟습니다. 검색어중 아이패드 케이스-아이폰 케이스, 흰색 드레스-노란색 드레스와 같이 비슷한 의미를 가진 검색어도 다수 존재하여 250보다 더 낮은수치에서 성능이 더 좋을수도 있습니다. 

---

LSA를 통해 얻고자 하는 것은 검색어 문서와 검색결과 문서의 연관성 정보를 가진 행렬 이므로 [TruncatedSVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)에서 계산 해주는 행렬을 그대로 이용하겟습니다. 좋은 정보인 유사도를 hstack으로 쌓아주고 [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)로 scale을 조정해서 LogisticRegression을 사용하였습니다.
(SVD와 similarity의 range가 다르기때문에 StandardScaler를 사용하였습니다.)

**LogisticRegression 코드**
    
```python
svd = TruncatedSVD(n_components = 250)
X_svd_train = svd.fit_transform(X_train, y)
X_svd_dev = svd.transform(X_dev)

X_stacked_train = hstack([X_svd_train, X_sim_train])
X_stacked_dev = hstack([X_svd_dev, X_sim_dev])
    
scl = StandardScaler(with_mean=False)
X_scaled_train = scl.fit_transform(X_stacked_train)
X_scaled_dev = scl.transform(X_stacked_dev)
    
lr = LogisticRegression()
lr.fit(X_scaled_train, y)
    
# kappa score = 0.50225
print(metric.quadratic_weighted_kappa(y_dev, lr.predict(X_scaled_dev)))
```

결과 kappa score는 0.50225점으로 내려갔습니다. 하이퍼 파라미터 최적화를 하지않은것을 감안하더라도 score 감소 폭이 큽니다. target이 이산형 멀티클래스인 점과 regression에서는 **i.i.d**를 만족하면서 target에 영향을 주는 feature선택이 중요한데, 연관성이 높으면 좋은 점수를 주는 모델링에서 로지스틱 회귀모형은 잘못된 선택 이였습니다. SVD를 거치면 유사한 데이터끼리 가까운 위치로 접근하게 되므로 **SVM**을 사용하기로 했습니다.  

### SVM

텍스트 문서가 TF-IDF, SVD, Scaler를 거치면서 252차원 벡터가 되었습니다. 즉, 문서를 252차원 유클리드 공간에서 하나의 점으로도 볼수 있습니다. SVM은 이런 점들을 나누는 경계선을 찾는 방법입니다. 경계선을 기준으로 클래스를 구분하기 때문에 비슷한 데이터가 뭉쳐있고 다른 클래스끼리 거리가 멀수록 성능이 좋습니다.

---

**SVM 설명**

SVM(Support Vector Machine)를 이해하기 위해서는 선형대수학과 미적분에 대한 배경지식이 필요로 합니다. SVD와 마찬가지로 수학적인 성질이나 자세한 증명은 넘어가고 간략한 설명과 왜, 어떻게 사용하는지에 초점을 맞추겟습니다.
    
SVM은 벡터를 클래스별로 나누는 경계선을 찾는 방법입니다. 각 2차원부터 252차원까지 경계선을 찾기 때문에 총 251개의 경계선을 찾는 방법이라고 생각할 수 있습니다. 그렇기 때문에 SVM은 target이 이산형 멀티클래스인 문제에서 성능이 잘나오는 모형입니다. 
    
SVM에서 중요한것은 마진과 커널입니다. 마진은 경계선을 찾을때 예외적인 상황을 얼마나 유연하게 처리할것인지를 정하고 커널은 경계선이 선형이 아닌 곡선으로 클래스를 나눌수 있게 해줍니다. 
    
---


**SVM 코드**
    
sklearn의 [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)을 사용하여 svm을 적용합니다.
    
```python
svd = TruncatedSVD(n_components = 250)
X_svd_train = svd.fit_transform(X_train, y)
X_svd_dev = svd.transform(X_dev)

X_stacked_train = hstack([X_svd_train, X_sim_train])
X_stacked_dev = hstack([X_svd_dev, X_sim_dev])
    
scl = StandardScaler(with_mean=False)
X_scaled_train = scl.fit_transform(X_stacked_train)
X_scaled_dev = scl.transform(X_stacked_dev)
    
svm = SVC(C=100)
svm.fit(X_scaled_train, y)
svm_pred = svm.predict(X_scaled_dev)

# kappa score = 0.653517
print(metric.quadratic_weighted_kappa(y_dev, svm_pred))
```

kaggle discussion에서 싱글 모델의 벤치마크 점수인 kappa score = **0.653517**점까지 올라갔습니다. cross validation을 통해 얻은 값은 아니지만 꽤 괜찮은 모델이 되었다고 생각되어 결과에 대한 해석을 해보겠습니다.

## 모델링결과해석

### 문제점

모델의 문제점을 파악 하기 위해서 data를 train, dev set으로 나누어 모델에 적용한뒤 점수차를 보는것이 도움이 됩니다. 하지만 목표 점수인 kappa score로는 모델이 얼마나 적합한지 얻을수 있는 정보가 한정적입니다. 따라서 추가적인 여러 지표들을 함께 해석할 필요가 있습니다. rmse, confusion matrix, roc curve를 함께 보겠습니다.

|      |train    |dev      |
|------|---------|---------|
|kappa |0.900322 |0.653517 |
|rmse  |0.439018 |0.783064 |

(rmse는 낮을수록 좋은값입니다.)train/dev 점수차가 크고 에서 kappa score가 0.9이상이라는 점은 **분산**이 높고 **오버피팅**이 있는것으로 해석됩니다. dev set의 kappa score를 보니 편차는 생각보다 좋아보입니다.

confusion matrix : 만족도가 2-3, 3-4 구간에서 굉장히 헷갈려합니다.

|   |1  |2  |3  |4  |           
|---|---|---|---|---|           
|1  |35 |14 |8  |5  |           
|2  |19 |57 |25 |40 |           
|3  |8  |**34** |42 |**98** |           
|4  |6  |17 |**42** |566|           

classification report : 만족도 4일때를 제외하곤 모두 낮은 수치입니다. 특히 3점일때는 분류를 못한다고 볼 수 있습니다.

|             |precision  |recall  |f1-score  |support  |
|-------------|-----------|--------|----------|---------|
|1            |0.51       |0.56    |0.54      |62       |
|2            |0.47       |0.40    |0.43      |141      |
|3            |**0.36**       |**0.23**    |**0.28**      |182      |
|4            |0.80       |0.90    |0.84      |631      |
|accuracy     |           |        |0.69      |1016     |
|macro avg    |0.53       |0.52    |0.52      |1016     |
|weighted avg |0.66       |0.69    |0.67      |1016     |

roc curve를 보면 만족도가 3인 경우 분류 실패가 확실히 보입니다. 만족도간 유사도와 재현율을 차이를 보아 **데이터 비대칭**이 원인으로 해석됩니다. 한편으론 데이터의 특성상 2, 3점의 구분이 모호한것도 원인이 될 수 있다는 생각도 듭니다. 

<img src="./img/roc.PNG"> 

분류에 실패한 데이터들을 직접 눈으로 보고 이해하는것도 중요한 작업입니다만, 기본적으로 머신러닝 모델을 적용하고자 할때는 모든 데이터를 사람이 직접 확인 하기 힘든 경우이기 때문에 모델이 가장 헷갈려하는 만족도 2-3, 3-4 구간 데이터를 보았습니다. 틀린 데이터를 직접보니 제가 만족도 점수를 매긴다면 모델이 예측한 값과 같은 만족도를 줄것으로 생각되는 데이터도 더러 있습니다. 스스로 평가자가 되어 데이터에 직접 만족도를 매겨보도록 하겟습니다. (데이터에 있는 제품들은 실제로 존재하기도 하여 구글, 아마존 이베이를 참고하였습니다.) 제가 매긴 만족도 평가와 모델이 판단한 만족도는 12개중 4개의 데이터가 일치하였습니다. 만족도는 사람마다 다르게 느껴지기 때문에 어느정도 분산은 감안해야 한다고 생각했습니다. 이런 방식으로 문제점을 찾는 분석을 human level error analysis라고 합니다. 


|query                  |product_title                                                 |svm_pred_dev |my_pred |y_dev |         
|-----------------------|-----------------------------------------------------------------------------------|--|--|--|        
|gold dress             |Barbie Collector BMFC Black and Gold Dress Barbie Doll                             |3 |2 |2 |         
|spiderman              |Ruff Hewn Boys' 8-20 Spiderman Glow In The Dark Graphic Tee                        |3 |**3** |2 | 
|harleydavidson         |The Harley-Davidson Legend Poster Print by Libero Patrignani (27 x 39)             |3 |2 |3 | 
|keurig coffee maker    |Green Mountain Coffee Hazelnut K-Cups for Keurig Brewers (Box of 96)               |2 |**2** |3 | 
|dc shoes black         |Toddler Boy's Batman Light Up Sneakers - Black                                     |2 |3 |2 | 
|nike flip flops        |Disney Frozen Toddler Girl's Beach Flip-flop Sandal                                |2 |**2** |3 | 
|polaroid camera        |Polaroid Cube Waterproof Case/Suction Camera and Camcorder Mount - Black (POLC3WSM)|4 |3 |3 | 
|flannel sheets         |Pointehaven Heavy Weight Metro Printed Flannel Sheet Set                           |4 |**4** |3 | 
|eye cream              |Yes To Cucumbers Cucumbers Soothing Eye Gel - 1.01 floz                            |4 |3 |3 | 
|leather mens briefcase |Leather Double-Compartment Briefcase  Brown                                        |3 |4 |4 | 
|dresses for women      |Faded Glory Henley Maxi Dress With Rope Belt                                       |3 |4 |4 | 
|sweater dress          |Women's French Terry Draw Waist Dress - Cherokee                                   |3 |4 |4 | 


### 개선안

모델의 문제점을 분석해본결과 1. **높은 분산과 오버피팅** 2. **비대칭 데이터** 문제로 추려집니다.

#### 높은분산과오버피팅

높은 분산과 오버피팅을 줄이는 간단하면서 효과적인 방법은 **데이터를 더 추가** 하는 것입니다. 하지만 kaggle에서 제공한 데이터는 제한되어 있으므로 데이터를 추가하고 싶다면, data augmentation을 하여야 합니다. [EDA: Easy Data Augmentation techniques for boosting performance on text classification tasks](https://arxiv.org/abs/1901.11196)를 참고하여 Augmentation을 하겠습니다. EDA에서 소개한 data augmentation방법은 4가지가 있습니다. 1. 문장에서 단어를 랜덤하게 뽑아 동의어로 바꾸기 2. 랜덤한 단어를 랜덤한 위치에 넣는방밥, 3. 단어의 위치를 바꾸는 방법 4. 랜덤으로 선택된 단어를 삭제하는 방법

2, 4번 방법은 regularization 효과도 있다고 합니다. 

데이터 추가 이외의 방법으로는 **regularization**이 있습니다. clasification model이 SVM이므로 마진(C)값을 조정하면 될것 같습니다.

사실 모든 머신러닝 모델이 그렇듯 architecture 자체의 문제가 아니라면 **전처리**에 더욱 신경 쓰는것이 가장 확실하고 강력한 방법입니다. 하지만 실제 서비스를 개발하는것이 아니기 때문에 전처리는 적당한 선에서 끝내기로 합니다.

#### 비대칭데이터

비대칭 데이터 문제는 데이터 분석을 하면서 겪는 흔한 문제중 하나입니다. 꼭 자연어 처리 분야가 아니더라도 스팸 필터, 신용 카드 사기 검출, 암 검진과 같이 데이터 불균형은 자주 방생하며 현실에서 균일한 분포의 데이터를 얻을 확률또한 낮습니다. 기본적으로 검색 서비스를 제공할때 이미 만족도를 높게 하는것을 목표로 개발 되었을 것이기 때문에 비대칭 데이터는 필연적으로 보이기까지 합니다. 이러한 데이터들의 특징은 소수의 클래스를 올바르게 검출하는것이 중요하다는 것입니다. 한가지 좋은 소식은 흔한 문제인 만큼 개선안도 여러가지가 있습니다.
[](https://medium.com/lumiata/cross-validation-for-imbalanced-datasets-9d203ba47e8)

1. 샘플링  
데이터를 제거 또는 생성하는 방법으로 데이터 분포의 비대칭도를 낮추는 효과가 있습니다. **언더샘플링**, **오버샘플링**, **복합샘플링** 방법이 있습니다. 다만 자연어 처리 분야에서는 오버샘플링과 복합샘플링은 현실에서 존재할 수 없는 문서를 생성해내기도 하여 조심스럽게 적용하여야 합니다. 

2. 모델 변경  
비대칭 데이터에서 svm보다 성능이 조금더 잘 나온다고 알려진 모델을 사용하는것입니다. 일반적으로 **Decision Tree**를 기반으로 하는 방법들이 비대칭 데이터에서 성능이 잘 나온다고 알려져있습니다.


3. 평가 metric변경 
만족도가 4인 데이터가 전체의 60%이상을 차지하고 있으므로 모든 만족도가 4라고 예측하면 정확도가 60%는 나옵니다. 그래서 다수가 아닌 클래스의 정밀도와 재현율이 중요합니다. 이 둘을 모두 볼수있는 지표는 f1-score나 **area under the precision-recall curve**입니다. 여기서는 area under the precision-recall curve를 그래프와 함께 보겠습니다. 

## 모델 개선

데이터 증강, 샘플링 방법을 각각 적용한 방법과 함께 적용한 방법 3가지의 결과를 그래프와 함께 보겠습니다.

### SVMSMOTE 샘플링을한 결과입니다.

```
kappa  
train : 0.929231	dev : 0.651356
rmse
train : 0.422783	dev : 0.816898

confusion_matrix : 1016개중 345개를 틀림
 [[ 49  20   3   5]
  [ 13  68  31  36]
  [  6  29  59  80]
  [  7  34  81 495]]

classification_report :
               precision    recall  f1-score   support

           1       0.65      0.64      0.64        77
           2       0.45      0.46      0.45       148
           3       0.34      0.34      0.34       174
           4       0.80      0.80      0.80       617

    accuracy                           0.66      1016
   macro avg       0.56      0.56      0.56      1016
weighted avg       0.66      0.66      0.66      1016
```
데이터간 균형이 맞아지면서 모델의 성능이 조금 올랐습니다. 비대칭도를 샘플링으로 맞춰준 결과 모델이 햇갈려했던 3점과 검색서비스에서 중요한 1점의 성능이 향상되었습니다. 하지만 여전히 오버피팅이 존재합니다.

### augmentation을 하고 샘플링은 하지 않은 모델의 결과입니다.  

만족도 1에 대하여 8배 2,3에 대하여 4배 데이터 증강을 하였습니다.

```
kappa
train : 0.801718	dev : 0.636654
rmse
train : 0.720348	dev : 0.846484

confusion_matrix : 1016개중 351개를 틀림
 [[ 46  15   7   9]
  [ 21  68  37  22]
  [  9  39  65  61]
  [ 13  29  89 486]]

classification_report :
               precision    recall  f1-score   support

           1       0.52      0.60      0.55        77
           2       0.45      0.46      0.45       148
           3       0.33      0.37      0.35       174
           4       0.84      0.79      0.81       617

    accuracy                           0.65      1016
   macro avg       0.53      0.55      0.54      1016
weighted avg       0.67      0.65      0.66      1016
```

먼저 눈에 띄는점은 만족도 1점의 성능입니다. augmentation을 통해 데이터가 부족했던 1점 데이터의 비율을 높혀 비대칭도를 낮춘결과 성능이 향상되었습니다. 샘플링을 한후에도 비대칭도가 존재하긴 하지만 성능이 향상되었고 오버피팅이 개선될걸로 보아 augmentation이 유의미하게 작용한것으로 보입니다.

## 그리드서치

### 병렬처리

데이터를 증강, 오버샘플링하면 데이터가 증가하여 계산복잡도가 높은 SVM에서 시간이 오래걸리기 때문에 병렬처리를 통해 Cross-validation 계산 속도를 높혀야 합니다. 하지만 sklearn에서 제공하는 Pipeline은 val data에 oversampling을 포함하지 않는 기능이 구현되어 있지 않습니다. 어쩔 수 없이 sklearn의 내부 코드를 참고하여 병렬처리 cross-validation을 직접 구현하였습니다. (코드는 model.py gridsearchCV class를 참고해 주세요)

### 그리드서치 결과

data augmentation 과정에서 새로운 단어들이 추가되어 TF-IDF행렬의 크기가 늘어났고 data총량이 늘어났기때문에 SVD사이즈와 SVM의 C(마진) 조정에 중점을 두어 하이퍼 파라미터를 탐색해보았습니다. 다음은 그리드 서치를 통해 찾은 최종 모델의 하이퍼 파라미터 입니다.

```
tfv = text.TfidfVectorizer(min_df=7)
sim = similarlity_stack()
svd = TruncatedSVD(n_components = 250)
scl = StandardScaler(with_mean=False)
svmsmote = SVMSMOTE(k_neighbors=4, svm_estimator=SVC(C=10))
svm = SVC(C = 10, gamma='auto', kernel='rbf)
```
EDA는 논문에서 제안한 하이퍼 파라미터를 사용하였습니다.
```
augment.py --input=./data/eda/train_1.txt --num_aug=8 --alpha=0.05 augment.py --input=./data/eda/train_2.txt --num_aug=4 --alpha=0.05 augment.py --input=./data/eda/train_3.txt --num_aug=4 --alpha=0.05 augment.py --input=./data/eda/train_4.txt --num_aug=1 --alpha=0.01 
processing.py --input=./data/test.csv"
processing.py --input=./data/eda_train.csv
```

두가지 모델의 soft voting 방식으로 투표한 결과를 최종 모델로 결정하겠습니다. test set 기준 kappa score 0.3점이 향상되었습니다.

### 다른 모델들

TF-IDF기반 모델의 가장 큰 단점중 하나는 데이터를 추가 하여 훈련을 할때 TF-IDF행렬을 다시 계산하여야 한다는것 입니다. 특히 계산복잡도가 높은 SVM 까지 매번 데이터가 쌓이면서 처음부터 다시 모델을 훈련시키는것은 시간과 비용의 낭비가 심각하므로 매우 바보같은 방법입니다.  

이는 데이터가 추가되는 데로 훈련하여 성능을 높일수 있는 인공 신경망 기반 word embedding 방법으로 해결할 수 있어 보입니다. 하지만 word2vec과 Doc2Vec을 적용해 보았는데도 성능이 오히려 떨어졌습니다. pretrain된 모델을 사용하지 않았고 Co-occurrence 기반 딥러닝 모델을 훈련하기에 충분하지 않은 코퍼스양을 생각해보았을때 kaggle competition에서는 TF-IDF방법을 사용하여야 할 것 같습니다. 하지만 실제 서비스에서는 매번 새로운 데이터가 생성되므로 딥러닝 기반 word representation 방식이 훨신 이득으로 보입니다. (실험 과정은 git hub의 Word2Vec.ipynb에 나와있습니다.)  

본문에는 없지만 Pretrained BERT를 이용한 만족도 판별모델과 soft voting을 하여 최종 성적을 달성하였습니다.

## 결론

단일모델 kaggle submission 밴치마크 점수 달성만족도를 판별하는 모델을 만들었습니다. Neural Network기반 모델인 word2vec을 조금 적용해보았지만 성능의 개선은 볼수 없었습니다. 하지만 이후 BERT를 이용한 만족도 판별 모델로는 성능의 개선이 있었습니다. 개인적으로 추천시스템을 구현해보고 싶었지만 사용자 데이터가 없으므로 이후 같은데이터로 정보 검색 랭킹 시스템을 구현해 보겠습니다.

<img src=".\img\submission.PNG">